<a href="https://colab.research.google.com/github/Sriharshitha842/ML_LAB/blob/main/ML_lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Name : Sriharshitha Deepala**

**Reg No : 19BCD7246**

**Course Code : CSE3008**

**Lab No : 03**

# **ID3 DECISION TREE FOR CLASSIFICATION**

**1. PlayTennis.csv**

In [184]:
#importing data from csv file as DataFrame using pandas library
import pandas as pd
#reading data using pd.read_csv() method
df_given = pd.read_csv("/content/sample_data/PlayTennis.csv")
print("\n The given set of data are(from csv file):\n\n", df_given)


 The given set of data are(from csv file):

      Outlook Temperature Humidity    Wind PlayTennis
0      Sunny         Hot     High    Weak         No
1      Sunny         Hot     High  Strong         No
2   Overcast         Hot     High    Weak        Yes
3       Rain        Mild     High    Weak        Yes
4       Rain        Cool   Normal    Weak        Yes
5       Rain        Cool   Normal  Strong         No
6   Overcast        Cool   Normal  Strong        Yes
7      Sunny        Mild     High    Weak         No
8      Sunny        Cool   Normal    Weak        Yes
9       Rain        Mild   Normal    Weak        Yes
10     Sunny        Mild   Normal  Strong        Yes
11  Overcast        Mild     High  Strong        Yes
12  Overcast         Hot   Normal    Weak        Yes
13      Rain        Mild     High  Strong         No


In [185]:
df_tennis.keys()

Index(['Outlook', 'Temperature', 'Humidity', 'Wind', 'PlayTennis',
       'predicted'],
      dtype='object')

In [186]:
# Defining two method to calculate Entropy using formula from textbook.
# entropy method will evaluate -p/(p+n)log2(p/(p+n))- n/(p+n)log2 (n/(p+n))
def entropy(probs):  
    import math
    return sum( [-prob*math.log(prob, 2) for prob in probs] )

# This method take input list and calulates its probability
# and pass probability value to entropy method to calculate entropy 
def entropyOfGivenData(a_list):  
    from collections import Counter
    cnt = Counter(x for x in a_list)
    num_instances = len(a_list)*1.0
    probs = [x/num_instances for x in cnt.values()]
    return entropy(probs) 
    
# printing input data for which we are calculating entropy
print("\n  Calculating entropy for input data set:\n\n", df_given['PlayTennis'])

#calculating entropy of Play_Tennis list using entropyOfGivenData and storing value in totalEntropy
totalEntropy = entropyOfGivenData(df_given['PlayTennis'])

print("\n The total Entropy of Give Data Set is:",totalEntropy)


  Calculating entropy for input data set:

 0      No
1      No
2     Yes
3     Yes
4     Yes
5      No
6     Yes
7      No
8     Yes
9     Yes
10    Yes
11    Yes
12    Yes
13     No
Name: PlayTennis, dtype: object

 The total Entropy of Give Data Set is: 0.9402859586706309


In [187]:
def information_gain(df, attributeNameToSplit, targetAttribute, trace=0):
    df_split = df.groupby(attributeNameToSplit)
    nobs = len(df.index) * 1.0
    df_agg_ent = df_split.agg({targetAttribute : [entropyOfGivenData, lambda x: len(x)/nobs] })[targetAttribute]
    df_agg_ent.columns = ['Entropy', 'PropObservations']
    new_entropy = sum( df_agg_ent['Entropy'] * df_agg_ent['PropObservations'] )
    old_entropy = entropyOfGivenData(df[targetAttribute])
    return old_entropy - new_entropy


print(' Information gain for Outlook is :'+str( information_gain(df_given, 'Outlook', 'PlayTennis')),"\n")
print(' Information gain for Humidity is: ' + str( information_gain(df_given, 'Humidity', 'PlayTennis')),"\n")
print(' Information gain for Wind is:' + str( information_gain(df_given, 'Wind', 'PlayTennis')),"\n")
print(' Information gain for Temperature is:' + str( information_gain(df_given, 'Temperature','PlayTennis')),"\n")

 Information gain for Outlook is :0.2467498197744391 

 Information gain for Humidity is: 0.15183550136234136 

 Information gain for Wind is:0.04812703040826927 

 Information gain for Temperature is:0.029222565658954647 



In [188]:
def id3(df, target_attribute_name, attNames, default_class=None):
    
    from collections import Counter
    cnt = Counter(x for x in df[target_attribute_name])
    if len(cnt) == 1:
        return next(iter(cnt))  
    elif df.empty or (not attNames):
        return default_class  
    else:
        
        default_class = max(cnt.keys()) 
        gainz = [information_gain(df, attr, target_attribute_name) for attr in attNames] #
        index_of_max = gainz.index(max(gainz)) 
        best_attr = attNames[index_of_max]
        tree = {best_attr:{}} 
        remaining_attribute_names = [i for i in attNames if i != best_attr]
        
        
        for attr_val, data_subset in df.groupby(best_attr):
            subtree = id3(data_subset,
                        target_attribute_name,
                        remaining_attribute_names,
                        default_class)
            tree[best_attr][attr_val] = subtree
        return tree

In [189]:
attNames = list(df_given.columns)
print("List of all attributes in given data:", attNames) 
attNames.remove('PlayTennis') #removing target attribute name
print("List of attributes used for:", attNames)

List of all attributes in given data: ['Outlook', 'Temperature', 'Humidity', 'Wind', 'PlayTennis']
List of attributes used for: ['Outlook', 'Temperature', 'Humidity', 'Wind']


In [190]:
# using pprint to print tree(type:dict) in herarical manner
from pprint import pprint
tree = id3(df_given,'PlayTennis',attNames)
print("\n\nThe Resultant Decision Tree is :\n")
pprint(tree)
attribute = next(iter(tree))
print("Best Attribute :\n",attribute,"\n")
print("Tree Keys:\n",tree[attribute].keys())



The Resultant Decision Tree is :

{'Outlook': {'Overcast': 'Yes',
             'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}},
             'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}
Best Attribute :
 Outlook 

Tree Keys:
 dict_keys(['Overcast', 'Rain', 'Sunny'])


In [191]:
#classification accuracy
def classify(instance, tree, default=None):
    attribute = next(iter(tree))  
    if instance[attribute] in tree[attribute].keys():   
        result = tree[attribute][instance[attribute]]
        if isinstance(result, dict): 
            return classify(instance, result)
        else:
            return result
    else:
        return default

In [192]:
# Checking accuracy of all training data with given data
# here also considering test data and training data as given data
df_given['predicted'] = df_given.apply(classify, axis=1, args=(tree,'No') ) 
accuracy1=sum(df_given['PlayTennis']==df_given['predicted'] )*100/len(df_given)
print('\n Accuracy is: ',accuracy1,"%\n")

df_given[['PlayTennis', 'predicted']]


 Accuracy is:  100.0 %



,PlayTennis,predicted
0,No,No
1,No,No
2,Yes,Yes
3,Yes,Yes
4,Yes,Yes
5,No,No
6,Yes,Yes
7,No,No
8,Yes,Yes
9,Yes,Yes


**2. PlayTennisTest.csv**

In [210]:
import pandas as pd
import numpy as np

In [211]:
train_data_m = pd.read_csv('/content/sample_data/PlayTennisTest.csv')
train_data_m

,Outlook,Temperature,Humidity,Wind,PlayTennis
0,Sunny,Hot,Normal,Weak,?
1,Rain,Mild,High,Strong,?


In [212]:
def calc_total_entropy(train_data, label, class_list):
    total_row = train_data.shape[0] 
    total_entr = 0
    
    for c in class_list: 
        total_class_count = train_data[train_data[label] == c].shape[0] 
        total_class_entr = - (total_class_count/total_row)*np.log2(total_class_count/total_row) 
        total_entr += total_class_entr 
    
    return total_entr

In [213]:
def calc_entropy(feature_value_data, label, class_list):
    class_count = feature_value_data.shape[0]
    entropy = 0
    
    for c in class_list:
        label_class_count = feature_value_data[feature_value_data[label] == c].shape[0] #row count of class c 
        entropy_class = 0
        if label_class_count != 0:
            probability_class = label_class_count/class_count #probability of the class
            entropy_class = - probability_class * np.log2(probability_class)  #entropy
        entropy += entropy_class
    return entropy

In [215]:
def calc_info_gain(feature_name, train_data, label, class_list):
    feature_value_list = train_data[feature_name].unique() 
    total_row = train_data.shape[0]
    feature_info = 0.0
    
    for feature_value in feature_value_list:
        feature_value_data = train_data[train_data[feature_name] == feature_value] 
        feature_value_count = feature_value_data.shape[0]
        feature_value_entropy = calc_entropy(feature_value_data, label, class_list) 
        feature_value_probability = feature_value_count/total_row
        feature_info += feature_value_probability * feature_value_entropy 
        
    return calc_total_entropy(train_data, label, class_list) - feature_info 

In [216]:
def find_most_informative_feature(train_data, label, class_list):
    feature_list = train_data.columns.drop(label) 
                                      
    max_info_gain = -1
    max_info_feature = None
    
    for feature in feature_list:  
        feature_info_gain = calc_info_gain(feature, train_data, label, class_list)
        if max_info_gain < feature_info_gain: 
            max_info_gain = feature_info_gain
            max_info_feature = feature
            
    return max_info_feature

In [217]:
def generate_sub_tree(feature_name, train_data, label, class_list):
    feature_value_count_dict = train_data[feature_name].value_counts(sort=False) 
    tree = {} #sub tree or node
    
    for feature_value, count in feature_value_count_dict.iteritems():
        feature_value_data = train_data[train_data[feature_name] == feature_value] 
        
        assigned_to_node = False 
        for c in class_list: 
            class_count = feature_value_data[feature_value_data[label] == c].shape[0] 

            if class_count == count: 
                tree[feature_value] = c 
                train_data = train_data[train_data[feature_name] != feature_value] 
                assigned_to_node = True
        if not assigned_to_node: 
            tree[feature_value] = "?" 
            
    return tree, train_data

In [218]:
def make_tree(root, prev_feature_value, train_data, label, class_list):
    if train_data.shape[0] != 0: 
        max_info_feature = find_most_informative_feature(train_data, label, class_list) 
        tree, train_data = generate_sub_tree(max_info_feature, train_data, label, class_list) 
        next_root = None
        
        if prev_feature_value != None: 
            root[prev_feature_value] = dict()
            root[prev_feature_value][max_info_feature] = tree
            next_root = root[prev_feature_value][max_info_feature]
        else: #add to root of the tree
            root[max_info_feature] = tree
            next_root = root[max_info_feature]
        
        for node, branch in list(next_root.items()): 
            if branch == "?": 
                feature_value_data = train_data[train_data[max_info_feature] == node] 
                make_tree(next_root, node, feature_value_data, label, class_list) 
                

In [219]:
def id3(train_data_m, label):
    train_data = train_data_m.copy() 
    tree = {} 
    class_list = train_data[label].unique() 
    make_tree(tree, None, train_data_m, label, class_list) 
    return tree

In [220]:
tree = id3(train_data_m, 'PlayTennis')

In [221]:
def evaluate(tree, test_data_m, label):
    correct_preditct = 0
    wrong_preditct = 0
    for index, row in test_data_m.iterrows(): 
        result = predict(tree, test_data_m.iloc[index]) 
        if result == test_data_m[label].iloc[index]:
            correct_preditct += 1
        else:
            wrong_preditct += 1 
    accuracy = correct_preditct / (correct_preditct + wrong_preditct) 
    return accuracy

In [225]:
print(tree)

{'Outlook': {'Sunny': '?', 'Rain': '?'}}


In [224]:
test_data_m = pd.read_csv("/content/sample_data/PlayTennisTest.csv")
accuracy = evaluate(tree, test_data_m, 'PlayTennis')
test_data_m['predicted'] = test_data_m.apply(classify, axis=1, args=(tree,'No') ) 
print(test_data_m['predicted'])
print("Accuracy is ",accuracy)
test_data_m[['PlayTennis', 'predicted']]


0    ?
1    ?
Name: predicted, dtype: object
Accuracy is  1.0


,PlayTennis,predicted
0,?,?
1,?,?
